In [ ]:
# Proyecto Final - Preparación de Datos
# Módulo 3: Obtención y Preparación de Datos

# =============================
# Lección 1 - Uso de NumPy
# =============================

import numpy as np
import pandas as pd
from IPython.display import display

print("========== LECCIÓN 1: CREACIÓN DE DATOS CON NUMPY ==========\n")

np.random.seed(0)  # Para reproducibilidad
clientes = np.arange(1, 101)  # IDs de 100 clientes
montos = np.random.normal(loc=500, scale=120, size=100).round(2)  # Monto promedio de compra
edades = np.random.randint(18, 70, size=100)

# Guardar los arrays para reutilizar
np.save("clientes.npy", clientes)
np.save("montos.npy", montos)
np.save("edades.npy", edades)

print("Datos generados y guardados exitosamente.\n")

# =============================
# Lección 2 - Exploración con Pandas
# =============================

print("\n========== LECCIÓN 2: EXPLORACIÓN CON PANDAS ==========\n")

clientes = np.load("clientes.npy")
montos = np.load("montos.npy")
edades = np.load("edades.npy")

df = pd.DataFrame({
    "ID_Cliente": clientes,
    "Monto_Compra": montos,
    "Edad": edades
})

print("▶ Primeras 5 filas del DataFrame:")
display(df.head())

print("\n▶ Estadísticas descriptivas:")
display(df.describe())

print("\n▶ Clientes mayores de 60 años:")
display(df[df["Edad"] > 60])

df.to_csv("datos_clientes.csv", index=False)
print("\nArchivo CSV guardado como 'datos_clientes.csv'.")

# =============================
# Lección 3 - Carga desde múltiples fuentes
# =============================

print("\n========== LECCIÓN 3: INTEGRACIÓN DE FUENTES DE DATOS ==========\n")

df_csv = pd.read_csv("datos_clientes.csv")

# Crear y cargar archivo Excel ficticio
df_excel = pd.DataFrame({
    "ID_Cliente": np.arange(51, 151),
    "Frecuencia_Compra": np.random.randint(1, 20, 100)
})
df_excel.to_excel("frecuencias_clientes.xlsx", index=False)
df_excel = pd.read_excel("frecuencias_clientes.xlsx")

# Cargar tabla web desde Wikipedia
print("▶ Cargando tabla desde Wikipedia...")
url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
tablas = pd.read_html(url)
df_web = tablas[2] if len(tablas) > 2 else tablas[0]
print(f"Se encontraron {len(tablas)} tablas. Se utilizó la tabla {2 if len(tablas) > 2 else 0}.\n")

# Unificar datos CSV y Excel
df_unido = pd.merge(df_csv, df_excel, on="ID_Cliente", how="outer")
print("▶ Dataset unificado (primeras 5 filas):")
display(df_unido.head())

# =============================
# Lección 4 - Limpieza de datos
# =============================

print("\n========== LECCIÓN 4: LIMPIEZA DE DATOS ==========\n")

print("▶ Valores nulos antes de limpieza:")
print(df_unido.isnull().sum())

df_unido["Frecuencia_Compra"] = df_unido["Frecuencia_Compra"].fillna(df_unido["Frecuencia_Compra"].mean())

# Detección de outliers con IQR
Q1 = df_unido["Monto_Compra"].quantile(0.25)
Q3 = df_unido["Monto_Compra"].quantile(0.75)
IQR = Q3 - Q1
filtro = ~((df_unido["Monto_Compra"] < (Q1 - 1.5 * IQR)) | (df_unido["Monto_Compra"] > (Q3 + 1.5 * IQR)))
outliers_removidos = len(df_unido) - filtro.sum()
df_unido = df_unido[filtro]

print(f"\n▶ Outliers eliminados: {outliers_removidos}")
print("\n▶ Dataset limpio (primeras 5 filas):")
display(df_unido.head())

df_unido.to_csv("datos_limpios.csv", index=False)

# =============================
# Lección 5 - Data Wrangling
# =============================

print("\n========== LECCIÓN 5: TRANSFORMACIÓN DE DATOS (WRANGLING) ==========\n")

print("▶ Filas duplicadas antes de eliminar:", df_unido.duplicated().sum())
df_unido = df_unido.drop_duplicates()

# Transformar tipos
df_unido["ID_Cliente"] = df_unido["ID_Cliente"].astype(int)

# Crear columna de segmento de edad
bins = [0, 30, 50, 70]
labels = ["Joven", "Adulto", "Mayor"]
df_unido["Segmento_Edad"] = pd.cut(df_unido["Edad"], bins=bins, labels=labels)

# Clasificar monto
clasificar = lambda x: "Alta" if x > 600 else "Media" if x > 400 else "Baja"
df_unido["Categoria_Monto"] = df_unido["Monto_Compra"].apply(clasificar)

print("▶ Dataset tras wrangling (primeras 5 filas):")
display(df_unido.head())

# =============================
# Lección 6 - Agrupamiento y pivoteo
# =============================

print("\n========== LECCIÓN 6: AGRUPAMIENTO Y PIVOTEO DE DATOS ==========\n")

print("▶ Agrupamiento por segmento de edad:")
agrupado = df_unido.groupby("Segmento_Edad")["Monto_Compra"].agg(["mean", "count"])
display(agrupado)

print("\n▶ Tabla dinámica (pivot):")
pivot = df_unido.pivot_table(index="Segmento_Edad", columns="Categoria_Monto", values="Monto_Compra", aggfunc="mean")
display(pivot)

print("\n▶ Reestructuración con melt:")
melted = df_unido.melt(id_vars=["ID_Cliente"], value_vars=["Monto_Compra", "Frecuencia_Compra"])
display(melted.head())

# Exportar resultados finales
df_unido.to_csv("dataset_final.csv", index=False)
df_unido.to_excel("dataset_final.xlsx", index=False)

print("\nArchivos exportados como 'dataset_final.csv' y 'dataset_final.xlsx'.")
print("\n========== PROCESO COMPLETO FINALIZADO ==========")


========== LECCIÓN 1: CREACIÓN DE DATOS CON NUMPY ==========

Datos generados y guardados exitosamente.


========== LECCIÓN 2: EXPLORACIÓN CON PANDAS ==========

▶ Primeras 5 filas del DataFrame:


,ID_Cliente,Monto_Compra,Edad
0,1,711.69,50
1,2,548.02,54
2,3,617.45,24
3,4,768.91,39
4,5,724.11,49



▶ Estadísticas descriptivas:


,ID_Cliente,Monto_Compra,Edad
count,100.000000,100.000000,100.000000
mean,50.500000,507.176300,43.850000
std,29.011492,121.555235,14.721231
min,1.000000,193.640000,18.000000
25%,25.750000,422.735000,29.750000
50%,50.500000,511.290000,45.000000
75%,75.250000,588.447500,57.000000
max,100.000000,772.370000,69.000000



▶ Clientes mayores de 60 años:


...

/tmp/ipython-input-13-1201038064.py:142: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df_unido.pivot_table(index="Segmento_Edad", columns="Categoria_Monto", values="Monto_Compra", aggfunc="mean")


Categoria_Monto,Alta,Baja,Media
Segmento_Edad,,,
Joven,630.293333,353.710000,494.687857
Adulto,694.255714,326.453750,492.918947
Mayor,680.788000,344.302857,514.482609



▶ Reestructuración con melt:


,ID_Cliente,variable,value
0,1,Monto_Compra,711.69
1,2,Monto_Compra,548.02
2,3,Monto_Compra,617.45
3,4,Monto_Compra,768.91
4,5,Monto_Compra,724.11



Archivos exportados como 'dataset_final.csv' y 'dataset_final.xlsx'.

========== PROCESO COMPLETO FINALIZADO ==========


In [22]:
# Último paso: Exportar y descargar los resultados finales

# 1. Instalar openpyxl para guardar archivos Excel
!pip install openpyxl --quiet

# 2. Importar módulos necesarios
import pandas as pd
from google.colab import files
import os
import time

# 3. Exportar archivo CSV
df_unido.to_csv("dataset_final.csv", index=False)
print("Archivo CSV guardado correctamente.")

# 4. Exportar archivo Excel de forma segura
try:
    with pd.ExcelWriter("dataset_final.xlsx", engine="openpyxl") as writer:
        df_unido.to_excel(writer, index=False)
    print("✅ Archivo Excel guardado correctamente.")
except Exception as e:
    print("❌ Error al guardar el archivo Excel:", e)

# 5. Mostrar archivos disponibles en el entorno de trabajo
print("\nArchivos generados en el entorno actual:")
print(os.listdir())

# 6. Esperar unos segundos para asegurar que todo esté guardado
time.sleep(2)

# 7. Descargar archivos si existen
if os.path.exists("dataset_final.csv"):
    print("\n⬇Descargando archivo CSV...")
    files.download("dataset_final.csv")
else:
    print("El archivo CSV no se encontró.")

if os.path.exists("dataset_final.xlsx"):
    print("⬇Descargando archivo Excel...")
    files.download("dataset_final.xlsx")
else:
    print("El archivo Excel no se encontró.")




Archivo CSV guardado correctamente.
✅ Archivo Excel guardado correctamente.

Archivos generados en el entorno actual:
['.config', 'dataset_final.csv', 'edades.npy', 'dataset_final.xlsx', 'frecuencias_clientes.xlsx', 'datos_limpios.csv', '.ipynb_checkpoints', 'datos_clientes.csv', 'clientes.npy', 'montos.npy', 'sample_data']

⬇Descargando archivo CSV...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇Descargando archivo Excel...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>